In [1]:
qpath = r'../../' # path to quetzal here
data = r'inputs/'
import sys
sys.path.insert(0, qpath)
# import class
from quetzal.io.gtfs_reader import importer

SQLalchemy is not installed. No support for SQL output.


### Read GTFS

In [2]:
feed = importer.GtfsImporter(path=data + r'paris_ratp_full', dist_units='m')
feed = feed.clean()
feed.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,"[2630435, 2650340, 12657225, 2679730, 2624437,..."
2,timezone,CET
3,start_date,20181003
4,end_date,20190101
5,num_routes,989
6,num_trips,989
7,num_stops,22352
8,num_shapes,0
9,num_frequencies,0


In [3]:
len(feed.trips)

989

In [4]:
feed.map_stops(
    feed.stops.stop_id.head(10000)  # IDs of the stops to plot
)

### Restrict to given dates

This restricts the feed to only trips and associated data running in one of the given dates.  
The calendar table is emptied and replaced by one calendar_dates row for each service running at one of these dates.

In [5]:
feed_r = feed.restrict(
    dates=['20181106', '20181111'] # the dates must be within the feed start and end dates
)
len(feed.trips)

989

In [6]:
feed_r.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,"[2630435, 2650340, 12661792, 2679416, 2616573,..."
2,timezone,CET
3,start_date,20181106
4,end_date,20181111
5,num_routes,330
6,num_trips,330
7,num_stops,6755
8,num_shapes,0
9,num_frequencies,0


In [7]:
feed_r.calendar  # Empty dataframe

In [8]:
print(len(feed_r.calendar_dates))
feed_r.calendar_dates.head()

222


,service_id,date,exception_type
0,2630435,20181106,1
1,2650340,20181106,1
2,12661792,20181106,1
3,2636085,20181106,1
4,2672264,20181106,1


In [9]:
feed_r.map_stops(
    feed_r.stops.stop_id
)

### Simplify services

Once the feed is restricted on a few dates only, it is possible to simplify its services by grouping those operating on the same dates.  
/!\ This makes GTFS edition more complicated afterwards as different trips will run on the same services!

In [10]:
feed_r.group_services()

In [11]:
feed_r.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,"[2630435, 2624437, 2699953]"
2,timezone,CET
3,start_date,20181106
4,end_date,20181111
5,num_routes,330
6,num_trips,330
7,num_stops,6755
8,num_shapes,0
9,num_frequencies,0


In [12]:
feed_r.calendar

In [13]:
feed_r.calendar_dates

,service_id,date,exception_type
0,2630435,20181106,1
120,2699953,20181106,1
122,2624437,20181111,1
220,2699953,20181111,1


Only 3 services remain: one operating on 20181106, one operating on 20181111 and one operating on both.

In [14]:
feed_r.map_stops(
    feed_r.stops.stop_id
)

### Restrict to a given service

In [15]:
# Pick one service running on the 2018-11-11
s_id = feed_r.calendar_dates[(feed_r.calendar_dates['date']=='20181111')].iloc[0]['service_id']
s_id

'2624437'

In [16]:
feed_s = feed_r.restrict(
    service_ids=[s_id]
)

In [17]:
feed_s.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,[2624437]
2,timezone,CET
3,start_date,20181111
4,end_date,20181111
5,num_routes,147
6,num_trips,147
7,num_stops,3425
8,num_shapes,0
9,num_frequencies,0


In [18]:
feed_s.calendar_dates

,service_id,date,exception_type
122,2624437,20181111,1


In [19]:
feed_s.map_stops(
    feed_s.stops.stop_id
)

### Restrict to a given time range

In [26]:
time_range = ['06:00:00', '09:00:00']  # time format must be HH:MM:SS
feed_t = feed.restrict(time_range=time_range)

In [27]:
feed_t.trips

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,shape_id
0,1025209,2689845,126898451022105,NaN,1,1,NaN
46,1030436,2679738,126797380938142,NaN,1,0,NaN
73,1086484,2616690,126166901084402,NaN,1,0,NaN
31317,1208812,2635684,1026356841208002,NaN,10,0,NaN
31487,1227999,2636085,126360850914553,NaN,1,0,NaN
...,...,...,...,...,...,...,...
434897,914412,2636162,126361620914186,NaN,1,0,NaN
434953,933332,2669393,126693931589777,NaN,1,0,NaN
444225,942178,2679208,126792080941182,NaN,1,0,NaN
444464,964247,2650595,126505950962599,NaN,1,0,NaN


In [22]:
feed_t.describe()

,indicator,value
0,agencies,[RATP (100)]
1,running_services,[2624437]
2,timezone,CET
3,start_date,20181111
4,end_date,20181111
5,num_routes,13
6,num_trips,13
7,num_stops,299
8,num_shapes,0
9,num_frequencies,0


In [23]:
feed_t.map_stops(feed_t.stops.stop_id)